
<div style="line-height:2px;border: solid orange">
    <p>
    <p style="color:blue;font-family:arial;text-align:right;font-size:20"> Visión por Computador &nbsp;&nbsp;
    <p style="color:blue;font-family:arial;text-align:right;font-size:16"> Master en Ciencias de Datos e Ingeniería de Ordenadores   &nbsp;&nbsp;
  <p style="color:blue;font-family:arial;text-align:right;font-size:16"> Rosa Mª. Rodríguez Sánchez   &nbsp;&nbsp;
    <p style="color:blue;font-family:arial;text-align:right;font-size:10"> Dpto. Ciencias de la Computación e Inteligencia Artificial. &nbsp;&nbsp;  
    <p style="color:blue;font-family:arial;text-align:right;font-size:10"> ETSIIT. Universidad de Granada   &nbsp;&nbsp;
        <p>
</div>

# Práctica 5 (Parte III). Transformada Wavelet

## Introducción
La transformada wavelet nos permite representar una imagen es un espacio escala-orientación. La escala representa la información de un determinado tamaño. Por ejemplo si la escala es baja veremos estructuras pequeñas, p.e información de cambios rápidos en intesidad, como son las aristas. La información de orientación te permite ver las estructuras que están en una determinada orientación. Para tal fin la transformada wavelet usa una función wavelet, denominada la función wavelet madre y una función de suavizado denominada de aproximación. Cuando aplicamos la transformada wavelet obtenemos dos tipos de informaciones: la de detalle y la de aproximación. La de detalle es la información que representa cambios rapidos. Y la aproximación es una versión de la imagen original pero con menor detalle, una versión suavizada. Así para una imagen aplicando una vez la transformada wavelet obtendremos una información de aproximación, informacion de detalle en horizontal, vertical y diagonal. Todas estas informaciones son imágenes de dimensiones la mitad de la original. Se puede aplicar otra vez sobre la aproximación la transformada. 
El numero de veces que se aplica esta descomposición se denomina el numero de niveles o escalas. 
En los ejemplos que veremos a continuación usaremos la libreria <a link= https://pywavelets.readthedocs.io/en/latest/install.html>PyWavelets </a>. Para instalar esta librearia podéis usar el siguiente comando:

In [ ]:
!pip install PyWavelets

En el siguiente código aplicamos la transformada wavelet a una imagen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pywt
import pywt.data

original = cv2.imread('miki_pluto.jpg',0)
# Wavelet transform 
titles = ['Approximación (LL)', ' Detalle Horizontal (LH)',
          'Detalle Vertical (HL)', 'Detalle Diagonal (HH)']
coeffs2 = pywt.dwt2(original, 'db1')
LL, (LH, HL, HH) = coeffs2
fig = plt.figure(figsize=(12, 12))
for i, a in enumerate([LL, LH, HL, HH]):
    ax = fig.add_subplot(1, 4, i + 1)
    ax.imshow(a, interpolation="nearest", cmap=plt.cm.gray)
    ax.set_title(titles[i], fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])
#Recuperar la imagen a partir de los coeficientes
rec=pywt.idwt2(coeffs2, 'db1')
plt.figure(figsize=(6, 6))
plt.imshow(rec,'gray'),plt.title('Recuperada')
    

La información se colocaría en una imagen de la siguiente forma

                            -------------------
                            |        |        |
                            | (LL)   |   (LH) |
                            |        |        |
                            -------------------
                            |        |        |
                            | (HL)   |   (HH) |
                            |        |        |
                            -------------------

Las bases  (o familias) que podemos usar para aplicar la transformada wavelet se muestran con el siguiente código

In [ ]:
 pywt.families()

Una familia concreta de wavelets

In [ ]:
pywt.wavelist('db')

Ver los filtros concretos de una transformada

In [ ]:
def format_array(arr):
    return "[%s]" % ", ".join(["%.14f" % x for x in arr])

wavelet = pywt.Wavelet('db1')
print(wavelet)
print("Filtros para analisis ",format_array(wavelet.dec_lo), format_array(wavelet.dec_hi))
print("Filtros para recuperacion ",format_array(wavelet.rec_lo), format_array(wavelet.rec_hi))

### Aplicar la transformada con diferentes niveles.

In [ ]:
def imadjust(img, Out=(0,1.0), gamma=1.0):
    
    max_in = np.max(img)
    min_in = np.min(img)
    
    low_out, high_out = Out
 
    
    low_out *= 255.0
    high_out *= 255.0    
    
    k = (high_out - low_out) / (max_in - min_in)
         
    table = np.array([((i / 255.0) ** gamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    h,w = img.shape[:2]
    imgOut = np.zeros((h,w),'float')
    
    for r in range(h):
        for c in range(w):
            res = int(k*(img[r,c]-min_in) + low_out)
            imgOut[r,c] = table[res]
               
    return imgOut


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pywt
from pywt._doc_utils import wavedec2_keys, draw_2d_wp_basis

original = cv2.imread('barbara.png',0)
print("original", original.shape)
shape = original.shape
label_levels = 3  # how many levels to explicitly label on the plots

max_lev = 3
fig, axes = plt.subplots(1, 3, figsize=[14, 8])

for level in range(1,max_lev + 1):
    
        
    # plot subband boundaries of a standard DWT basis
    draw_2d_wp_basis(shape, wavedec2_keys(level), ax=axes[ level-1],
                     label_levels=label_levels)
    axes[ level-1].set_title('{} level\ndecomposition'.format(level))




le=2
coeffs = pywt.wavedec2(original, 'db2',mode='periodization',level=le)
tam_i = coeffs[0].shape[0]
tam_j = coeffs[0].shape[1]
#Contabilizaos cuantas filas y columnas tiene la transformada
for i in range(1,len(coeffs)):
    print(coeffs[i][0].shape)
    tam_i += coeffs[i][0].shape[0]
    tam_j += coeffs[i][0].shape[1] 
    
    
    
print( "Total de filas ", tam_i," Columnas ",tam_j)
#Colocamos en img_l la tranformada wavelet para visualizar
img_l=np.zeros((tam_i,tam_j),dtype='float32')
dim_i=coeffs[0].shape[0]
dim_j=coeffs[0].shape[1]
ini_i=0;
ini_j=0;
#Colocamos la aproximacion ajustando el contraste
cl = imadjust(coeffs[0], Out=(0,1), gamma=1)
img_l[ini_i:ini_i+dim_i,ini_j:ini_j+dim_j]=cl

#Para el resto de bandas Hlm LH y HH 
for i in range(1,len(coeffs)):
    
    k_i=coeffs[i][0].shape[0]
    k_j=coeffs[i][0].shape[1] 
    
    
    cl = imadjust(coeffs[i][0], Out=(0,1), gamma=1)
    img_l[ini_i:ini_i+k_i,ini_j+dim_j:ini_j+dim_j+k_j]=cl
    cl = imadjust(coeffs[i][1], Out=(0,1), gamma=1)
    img_l[ini_i+dim_i:ini_i+dim_i+k_i,ini_j:ini_j+k_j]=cl
    cl = imadjust(coeffs[i][2], Out=(0,1), gamma=1)    
    img_l[ini_i+dim_i:ini_i+dim_i+k_i,ini_j+dim_j:ini_j+dim_j+k_j]=cl
    dim_i+=k_i
    dim_j+=k_j
plt.figure(figsize=(8,8))
plt.imshow(img_l,'gray')         
#Recuperar la informacion 
rec=pywt.waverec2(coeffs, 'db2',mode='periodization')
plt.figure()
plt.imshow(rec,'gray')         


### Ejercicio. Aproximación a la Compresión
Realizar sobre la imagen barbara una descomposición wavelet usando bior3.7 con tres niveles. Fijado un porcentaje de coeficientes, por ejemplo 10% del total de coeficientes, que  indica el porcentaje de coeficientes que nos quedamos de entre todos los coeficientes wavelets de la descomposición. El resto los ponemos a cero. Los coeficientes que mantenemos son los que tiene mayor magnitud.
Varias el porcentaje a (10,20,30,40,50,70,80 90) para:
<ul>
<li>Obtener una gráfica en la que en el eje X tenemos razón de compresión y en el eje Y el valor de PSNR entre la imagen original y la imagen reconstruida.
<li>Obtener una gráfica en la que se muestra el porcentaje de coeficientes que se han mantenido por banda. 
</ul>    